In [33]:
# %reset -sf

In [35]:
import sys
import numpy as np
import pandas as pd
import lightgbm as lgb

DEBUG = True

In [36]:
if DEBUG:
    with open("g0", "r") as f:
        lines = f.readlines()
else:
    sys.stdin.readlines()

# Read data

In [37]:
n,m = [int(x) for x in lines[0].split()]
train_data = []
for line in lines[1:n+1]:
    lst = [list(map(int,x.split(":"))) for x in line.split()]
    train_data.append([[x[0] for x in lst], [x[1] for x in lst]])
targets = []
for line in lines[n+1:n+1+n]:
    targets.append(int(line))
test_data = []
for line in lines[n+1+n:n+1+n+m]:
    lst = [list(map(int,x.split(":"))) for x in line.split()]
    test_data.append([[x[0] for x in lst], [x[1] for x in lst]])

# Feature engineering

In [61]:
def process_data(dataset):
    processed = []
    for loc,time in dataset:
        time = [0] + [b-a for a,b in zip(time, time[1:])]*200  # take time diff and cycle
        time = time[:200]
        loc = loc*200  # cycle locations
        loc = loc[:200]
        processed.append(time + loc)
    return processed

In [62]:
train_data_p = process_data(train_data)
test_data_p = process_data(test_data)
target_train = np.array(targets)
df_train = pd.DataFrame(train_data_p)
df_test = pd.DataFrame(test_data_p)

In [40]:
if DEBUG:
    print(len(train_data_p), len(train_data_p[0]))
    print(train_data_p[0])  # 200 indexes and 200 time intervals

7000 400
[0, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033, 0, 11964, 18432, 15014, 15997, 19877, 12008, 17033,

In [41]:
eval_set = np.array([True if i < len(df_train)*0.2 else False for i in range(len(df_train))])
lgb_train = lgb.Dataset(df_train[~eval_set], target_train[~eval_set])
lgb_eval = lgb.Dataset(df_train[eval_set], target_train[eval_set], reference=lgb_train)
lgb_all = lgb.Dataset(df_train, target_train)

In [60]:
from sklearn.metrics import f1_score

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

if DEBUG:
    baseline_f1 = f1_score(target_train[eval_set], [1]*len(target_train[eval_set]))
    print("F1 baseline: {:.2f}, F1 target: {:.2f}".format(baseline_f1, baseline_f1*1.7))

F1 baseline: 0.34, F1 target: 0.58


In [59]:
params = {
    'objective': 'binary',
    'verbose': -1 if not DEBUG else 0,
}

gbm = lgb.train(params,
                lgb_train,
                feval=lgb_f1_score,
                valid_sets=lgb_eval,
                verbose_eval=DEBUG,
                early_stopping_rounds=5)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022350 seconds.
You can set `force_col_wise=true` to remove the overhead.
[1]	valid_0's binary_logloss: 0.452532	valid_0's f1: 0
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.413218	valid_0's f1: 0
[3]	valid_0's binary_logloss: 0.382345	valid_0's f1: 0
[4]	valid_0's binary_logloss: 0.357674	valid_0's f1: 0
[5]	valid_0's binary_logloss: 0.336086	valid_0's f1: 0.461538
[6]	valid_0's binary_logloss: 0.319166	valid_0's f1: 0.517766
[7]	valid_0's binary_logloss: 0.303869	valid_0's f1: 0.570743
[8]	valid_0's binary_logloss: 0.290569	valid_0's f1: 0.591549
[9]	valid_0's binary_logloss: 0.279109	valid_0's f1: 0.617849
[10]	valid_0's binary_logloss: 0.269669	valid_0's f1: 0.643016
[11]	valid_0's binary_logloss: 0.260966	valid_0's f1: 0.656455
[12]	valid_0's binary_logloss: 0.254015	valid_0's f1: 0.659436
[13]	valid_0's binary_logloss: 0.247061	valid_0's f1: 0.

In [44]:
pred_test = gbm.predict(df_test, num_iteration=gbm.best_iteration)

In [46]:
res = [int(x > 0.5) for x in pred_test]
if not DEBUG:
    print("\n".join(str(x) for x in res))

In [32]:
# !jupyter nbconvert --to script *.ipynb